### Inference Test

In [1]:
from __future__ import print_function

import os
import sys

import numpy as np
import tensorflow as tf

sys.path.extend([".", ".."])  # adds the folder from which you call the script
os.environ["CASAPOSE_INFERENCE"] = "True"

from casapose.data_handler.image_only_dataset import ImageOnlyDataset
from casapose.data_handler.vectorfield_dataset import VectorfieldDataset
from casapose.pose_estimation.pose_evaluation import poses_pnp
from casapose.pose_estimation.voting_layers_2d import CoordLSVotingWeighted
from casapose.pose_models.tfkeras import Classifiers
from casapose.utils.config_parser import parse_config

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

/usr/local/lib/python3.8/dist-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `ContrastNormalization()` is deprecated. Use `imgaug.contrast.LinearContrast` instead.
  warn_deprecated(msg, stacklevel=3)
/usr/local/lib/python3.8/dist-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `FrequencyNoiseAlpha()` is deprecated. Use `BlendAlphaFrequencyNoise` instead. FrequencyNoiseAlpha is deprecated. Use BlendAlphaFrequencyNoise instead. The order of parameters is the same. Parameter 'first' was renamed to 'foreground'. Parameter 'second' was renamed to 'background'.
  warn_deprecated(msg, stacklevel=3)
2023-04-06 19:31:39.273546: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 19:31:39

In [2]:
#opt = parse_config()
#parse_config()

In [3]:
class opt:
    modelname = 'casapose_c_gcu5'
    estimate_confidence = 1
    estimate_coords = 1
    confidence_regularization = 1
    object = 'obj_000001,obj_000005,obj_000006,obj_000008,obj_000009,obj_000010,obj_000011,obj_000016'

    no_points = 9
    save_debug_batch = 0

    imagesize = (448, 448)
    imagesize_test = (480, 640)
    crop_factor = 0.933333333
    pretrained = 1
    manualseed = 1237

    # losses
    mask_loss_weight = 1.0
    vertex_loss_weight = 0.5
    proxy_loss_weight = 0.015
    keypoint_loss_weight = 0.007
    filter_vertex_with_segmentation = 1
    filter_high_proxy_errors = 0
    use_bpnp_reprojection_loss = 0
    max_keypoint_pixel_error = 12.5

    # learning rate
    lr = 0.001
    lr_decay = 0.5
    lr_epochs_steps = 50,75,90

    # general
    gpuids = 0,1
    loginterval = 10
    epochs = 100
    batchsize = 4
    saveinterval = 5
    validationinterval = 1

    # data preprocessing
    workers = 0
    prefetch = 10

    # augmentation
    translation = 0
    rotation = 0
    noise = 0.0001
    brightness = 0.001
    contrast = 0.001
    saturation = 0.001
    hue = 0.001
    use_imgaug = 1

    # test
    min_object_size_test = 200
    write_poses = 0
    save_eval_batches = 0

    # output
    net = 'training_checkpoints'
    outf = 'train_casapose_8_16_objects'

    # config
    train_vectors_with_ground_truth = 1
    load_h5_weights = 0
    copy_weights_from_backup_network = 0
    copy_weights_add_confidence_maps = 0
    objects_in_input_network = 8
    objects_to_copy = 1
    objects_to_copy_list = 'config/objects_to_copy.csv'
    
    confidence_filter_estimates = 1
    confidence_choose_second = 0
    train_vectors_with_ground_truth = 0
    datatest_wxyz_quaterion = 0
    filter_test_with_gt = 0
    
    evalf = 'output'
    #datatest = 'import_data/test/test2'
    #datameshes = 'import_data/test/models'
    datatest = '/workspace/CASAPose/import_data/test/test'
    datameshes = '/workspace/CASAPose/import_data/test/models'
    data = ''
    datatest_path_filter = None
    color_dataset = 1
    train_validation_split = 0.9
    backbonename = 'resnet18'
    load_h5_weights = 1
    load_h5_filename = '../../../data/pretrained_models/result_w'

In [4]:
if not os.path.exists(opt.evalf):
    os.makedirs(opt.evalf)

In [5]:
checkpoint_path = opt.outf + "/" + opt.net

frozen_path = opt.outf + "/frozen_model"
img_out_path = opt.outf + "/control_output"

In [6]:
def create_dir(path):
    try:
        os.makedirs(path)
    except OSError:
        pass

In [7]:
create_dir(img_out_path)

In [8]:
# save the hyper parameters passed
with open(opt.evalf + "/header_eval.txt", "w") as file:
    file.write(str(opt))

In [9]:
# set the manual seed.
np.random.seed(opt.manualseed)
tf.random.set_seed(opt.manualseed)

In [10]:
test_dataset = None

device_ids = []
if len(opt.gpuids) == 1 and opt.gpuids[0] < 0:
    device_ids.append("/cpu:0")
else:
    device_ids.append("/gpu:{}".format(opt.gpuids[0]))
print(device_ids)

['/gpu:0']


In [11]:
objectsofinterest = [x.strip() for x in opt.object.split(",")]
no_objects = len(objectsofinterest)
separated_vectorfields = opt.modelname == "pvnet"

In [12]:
objectsofinterest

['obj_000001',
 'obj_000005',
 'obj_000006',
 'obj_000008',
 'obj_000009',
 'obj_000010',
 'obj_000011',
 'obj_000016']

In [13]:
testingdata = None
normal_imgs = [0.5, 0.5]

use_split = False
if opt.data == opt.datatest:
    print("split datasets with ratio {}".format(opt.train_validation_split))
    use_split = True

test_batches = 0

In [14]:
test_dataset = ImageOnlyDataset(root=opt.datatest)
testing_images, _ = test_dataset.generate_dataset(1)

2023-04-06 19:31:39.845849: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-06 19:31:39.846737: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-06 19:31:39.847471: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-06 19:31:39.848423: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-06 19:31:39.849131: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from S

In [15]:
ImageOnlyDataset(root=opt.datatest).imgs

['/workspace/CASAPose/import_data/test/test/000001/000001.jpg',
 '/workspace/CASAPose/import_data/test/test/000001/000002.jpg',
 '/workspace/CASAPose/import_data/test/test/000001/000003.jpg',
 '/workspace/CASAPose/import_data/test/test/000001/000004.jpg',
 '/workspace/CASAPose/import_data/test/test/000001/000005.jpg']

In [16]:
len(ImageOnlyDataset(root=opt.datatest))

5

In [17]:
ImageOnlyDataset(root=opt.datatest)[0]

{'path': '/workspace/CASAPose/import_data/test/test/000001/000001.jpg',
 'name': '000001'}

In [18]:
len(test_dataset)

5

In [19]:
len(testing_images)

5

In [20]:
testing_images

<BatchDataset element_spec=TensorSpec(shape=(None, 480, 640, None), dtype=tf.float32, name=None)>

In [21]:
test_dataset = VectorfieldDataset(
    root=opt.datatest,
    path_meshes=opt.datameshes,
    path_filter_root=opt.datatest_path_filter,
    color_input=opt.color_dataset,
    no_points=opt.no_points,
    objectsofinterest=objectsofinterest,
    noise=0.00001,
    data_size=None,
    save=opt.save_debug_batch,
    normal=normal_imgs,
    contrast=0.00001,
    brightness=0.00001,
    hue=0.00001,
    saturation=0.00001,
    random_translation=(0, 0),
    random_rotation=0,
    random_crop=False,
    use_validation_split=use_split,
    use_train_split=False,
    train_validation_split=opt.train_validation_split,
    output_folder=opt.evalf,
    visibility_filter=False,
    separated_vectorfields=(opt.modelname == "pvnet"),
)

/workspace/CASAPose/import_data/test/models/obj_000001/obj_000001.ply
/workspace/CASAPose/import_data/test/models/obj_000002/obj_000002.ply
/workspace/CASAPose/import_data/test/models/obj_000003/obj_000003.ply
/workspace/CASAPose/import_data/test/models/obj_000004/obj_000004.ply
/workspace/CASAPose/import_data/test/models/obj_000005/obj_000005.ply
/workspace/CASAPose/import_data/test/models/obj_000006/obj_000006.ply
/workspace/CASAPose/import_data/test/models/obj_000007/obj_000007.ply
/workspace/CASAPose/import_data/test/models/obj_000008/obj_000008.ply
/workspace/CASAPose/import_data/test/models/obj_000009/obj_000009.ply
/workspace/CASAPose/import_data/test/models/obj_000010/obj_000010.ply
/workspace/CASAPose/import_data/test/models/obj_000011/obj_000011.ply
/workspace/CASAPose/import_data/test/models/obj_000012/obj_000012.ply
/workspace/CASAPose/import_data/test/models/obj_000013/obj_000013.ply
/workspace/CASAPose/import_data/test/models/obj_000014/obj_000014.ply
/workspace/CASAPose/

In [22]:
# test_dataset = VectorfieldDataset(
#     root="/workspace/CASAPose/import_data/test/test", # Hardcoded param
#     path_meshes="/workspace/CASAPose/import_data/test/models", # Hardcoded param
#     path_filter_root=None, # Hardcoded param
#     color_input=True, # Hardcoded param
#     no_points=9, # Taken from config_8.ini
#     objectsofinterest=objectsofinterest,
#     noise=0.00001,
#     data_size=None,
#     save=False, # Hardcoded param
#     normal=normal_imgs,
#     contrast=0.00001,
#     brightness=0.00001,
#     hue=0.00001,
#     saturation=0.00001,
#     random_translation=(0, 0),
#     random_rotation=0,
#     random_crop=False,
#     use_validation_split=use_split,
#     use_train_split=False,
#     train_validation_split=None,
#     output_folder=None,
#     visibility_filter=False,
#     separated_vectorfields=(opt.modelname == "pvnet"),
# )

In [23]:
no_objects

8

In [24]:
len(test_dataset)

5

In [25]:
test_dataset[0]

{'path': '/workspace/CASAPose/import_data/test/test/000001/000001.jpg',
 'name': '000001.jpg',
 'txt': '/workspace/CASAPose/import_data/test/test/000001/000001.json',
 'seg': '/workspace/CASAPose/import_data/test/test/000001/000001.seg.png',
 'path_raw': '/workspace/CASAPose/import_data/test/test/000001'}

In [26]:
# print(len(test_dataset))
# print(len(testing_images))
# testingdata, test_batches = test_dataset.generate_dataset(
#     1, 1, 0, opt.imagesize_test, 1.0, 1, no_objects, shuffle=False
# )

#'epochs', 'prefetch', 'imagesize', 'cropratio', 'worker', and 'no_objects'

In [27]:
# Using ImageOnlyDataset

print(len(test_dataset))
print(len(testing_images))
testingdata, test_batches = ImageOnlyDataset(root=opt.datatest).generate_dataset(
   batchsize=1
)

5
5


In [28]:
mesh_vertex_array, mesh_vertex_count = test_dataset.generate_object_vertex_array()

print("testing data: {} batches".format(test_batches))

testing data: 5.0 batches


In [29]:
input_segmentation_shape = None
if opt.train_vectors_with_ground_truth is True:
    input_segmentation_shape = (
        opt.imagesize_test[0],
        opt.imagesize_test[1],
        1 + no_objects,
    )

In [30]:
input_segmentation_shape

In [31]:
height = opt.imagesize_test[0]
width = opt.imagesize_test[1]

In [32]:
width

640

In [33]:
CASAPose = Classifiers.get(opt.modelname)

In [34]:
ver_dim = opt.no_points * 2

In [35]:
ver_dim

18

In [36]:
if opt.modelname == "pvnet":
    ver_dim = ver_dim * no_objects

if opt.estimate_confidence:
    assert separated_vectorfields is not None, "confidence not compaitble with this model"
    ver_dim += opt.no_points

In [37]:
net = CASAPose(
    ver_dim=ver_dim,
    seg_dim=1 + no_objects,
    input_shape=(height, width, 3),
    input_segmentation_shape=input_segmentation_shape,
    weights="imagenet",
    base_model=opt.backbonename,
)

[<KerasTensor: shape=(None, 240, 320, 64) dtype=float32 (created by layer 'relu0')>, <KerasTensor: shape=(None, 120, 160, 64) dtype=float32 (created by layer 'stage2_unit1_relu1')>, <KerasTensor: shape=(None, 60, 80, 128) dtype=float32 (created by layer 'stage3_unit1_relu1')>, <KerasTensor: shape=(None, 60, 80, 256) dtype=float32 (created by layer 'stage4_unit1_relu1')>, <KerasTensor: shape=(None, 60, 80, 512) dtype=float32 (created by layer 'relu1')>]


In [38]:
checkpoint_prefix = os.path.join(checkpoint_path, "ckpt")
checkpoint = tf.train.Checkpoint(network=net)  # , optimizer=optimizer)

In [39]:
if opt.load_h5_weights is True:
    net.load_weights(frozen_path + "/" + opt.load_h5_filename + ".h5", by_name=True, skip_mismatch=True)

elif opt.net != "":
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path)).expect_partial()

for layer in net.layers:
    layer.trainable = False

In [40]:
net.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 480, 640, 3  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             [(None, 240, 320, 6  11186889    ['data[0][0]']                   
                                4),                                                               
                                 (None, 120, 160, 6                                               
                                4),                                                               
                                 (None, 60, 80, 128                                         

In [41]:
with open(opt.evalf + "/speed_eval.csv", "w") as file:
    file.write("batchid,speed \n")

In [42]:
def runnetwork(loader_iterator, batches, keypoints, camera_matrix, no_objects):
    @tf.function
    def test_step(img_batch):
        start_time = tf.timestamp()
        no_points = opt.no_points
        net_input = [tf.expand_dims(img_batch[0], 0)]
        output_net = net(net_input, training=False)  # all stages are present here
        output_seg, output_dirs, confidence = tf.split(output_net, [no_objects, no_points * 2, -1], 3)
        coordLSV_in = [output_seg, output_dirs, confidence]
        coords = CoordLSVotingWeighted(
            name="coords_ls_voting",
            num_classes=no_objects,
            num_points=no_points,
            filter_estimates=True,
        )(coordLSV_in)
        poses_est = poses_pnp(
            coords, output_seg, keypoints, camera_matrix, no_objects - 1, min_num=opt.min_object_size_test
        )
        tf.print(poses_est[0, 0, 0, 0, 0])  # access memory to get sure everything is done
        tf.print('tf.print=', output_seg)

        end_time = tf.timestamp()
        time_needed = end_time - start_time
        return time_needed

    speed = []
    for batch_idx in range(batches):
        img_batch = loader_iterator.get_next()
        time_needed = test_step(img_batch)
        speed.append(time_needed)
        with open(opt.evalf + "/speed_eval.csv", "a") as file:
            s = "{},{:.7f}\n".format(batch_idx + 1, time_needed)
            file.write(s)

    tf.print("average speed: {}".format(tf.reduce_mean(speed[10:])), summarize=-1)

    return loader_iterator

In [43]:
print("Test Batches: {} ".format(test_batches))

Test Batches: 5.0 


In [44]:
testingdata_iterator = iter(testingdata)

In [45]:
img_batch = testingdata_iterator.get_next()

In [46]:
len(img_batch)

1

In [47]:
keypoints_array = np.array([[[[[-9.7732497e-03,  3.6659201e-03, -1.4534000e-03],
          [ 3.6046398e+01, -1.4680500e+01, -4.5020599e+01],
          [-3.0289101e+01, -7.2402501e+00, -4.2632900e+01],
          [ 7.4425201e+00,  2.3966400e+01, -3.9362701e+01],
          [-4.3485899e+00,  3.6769500e+00,  4.5835800e+01],
          [ 6.2882501e-01, -3.6412800e+01, -2.7732599e+01],
          [-2.6754901e-01,  3.7588799e+01, -4.7640200e+00],
          [ 3.0029400e+01, -2.3939800e+01, -8.1097898e+00],
          [-2.8789900e+01, -1.9449200e+01, -9.0417604e+00]]],


        [[[ 1.6300200e-02, -2.3040799e-03, -1.1291500e-02],
          [ 5.5248199e+00,  5.4157101e+01, -9.6322701e+01],
          [-4.1018100e+00,  1.2732400e+01,  9.6678497e+01],
          [-9.1580000e+00, -4.1244202e+01, -8.7472397e+01],
          [ 7.3375401e+00,  9.0886101e+01, -1.1365300e+01],
          [-1.0262200e+01, -9.0547600e+01, -3.7563899e-01],
          [-4.7794201e+01,  1.6508699e+01, -5.6376900e+01],
          [ 4.8287998e+01,  2.4022501e+00, -6.2877899e+01],
          [ 4.6154099e+01,  1.1302400e+01,  4.9851101e+01]]],


        [[[ 1.7128000e-02, -4.5700101e-03, -5.3901700e-03],
          [ 2.0947300e+01, -6.1587502e+01, -5.4198200e+01],
          [-2.0933701e+01,  6.3563000e+01,  2.6130899e+01],
          [ 2.8901501e+01,  2.7392700e+01, -5.7568199e+01],
          [ 1.4403200e+00, -5.8665901e+01,  2.2473900e+01],
          [ 1.2946500e+01,  1.4082400e+01,  5.8292999e+01],
          [-2.8743299e+01,  1.6301001e+01, -5.2558300e+01],
          [-3.3441200e+01, -4.1310501e+01, -5.4232101e+01],
          [ 2.3869900e+01,  4.1699699e+01,  1.6587299e+01]]],


        [[[-2.4108901e-03, -6.2332200e-03, -6.3247699e-03],
          [ 1.1291000e+02, -3.4727199e+00,  9.2172699e+01],
          [-1.1182900e+02,  3.1709600e-02,  6.1154400e+01],
          [-6.2377201e+01,  1.0970700e+01, -1.0025700e+02],
          [ 4.2661201e+01, -2.4666700e+01, -9.9452499e+01],
          [ 1.0724100e+01, -3.5357201e+00,  1.0133300e+02],
          [-4.1970699e+01, -3.1155399e+01,  5.4645599e+01],
          [ 4.9310899e+00,  3.6434399e+01, -9.7123596e+01],
          [ 5.6840302e+01, -4.2665200e+00,  4.8058399e+01]]],


        [[[-3.4179699e-03, -9.8838797e-03,  3.9329501e-03],
          [ 4.9320702e+01,  6.2302999e+00, -4.0302898e+01],
          [-4.6246700e+01,  2.3396499e+00, -3.7502899e+01],
          [ 1.2448000e+01, -3.3365299e+01, -4.0734501e+01],
          [ 3.9640200e+00,  3.4297600e+01, -4.0923302e+01],
          [ 4.5272598e+01, -1.0067500e+00,  2.1399401e+01],
          [ 6.6833901e+00, -3.1548400e+00,  4.2783199e+01],
          [-2.3509399e+01, -2.7834400e+01, -1.9335600e+01],
          [-4.1355202e+01,  1.3988900e-01,  1.3391900e+00]]],


        [[[-1.7417900e-02, -4.2999300e-01, -1.3252300e-02],
          [-7.0443398e+01,  4.3526299e+01,  4.2999201e+00],
          [ 7.3233902e+01,  3.5586300e+01,  4.8644700e+00],
          [ 6.7131897e+01, -4.4466202e+01, -2.7725799e+00],
          [-7.0990898e+01, -3.6974701e+01, -1.3353300e+00],
          [-4.7924999e+01,  5.5036702e+00, -3.2836399e+01],
          [ 2.2584101e+01,  4.1242500e+01,  3.2724400e+01],
          [-2.4753901e+01, -4.0470100e+01,  3.2213699e+01],
          [ 4.7744598e+01,  4.2735401e-01, -3.1653799e+01]]],


        [[[ 9.9391900e-03, -1.1459400e-02,  6.8359398e-03],
          [-9.1147299e+00, -3.1402399e+01, -8.5777802e+01],
          [ 9.7676700e-01,  2.9348700e+00,  8.6390404e+01],
          [ 6.4356799e+00,  3.7870701e+01, -6.3978802e+01],
          [ 9.7071304e+00, -3.6640800e+01, -3.6885799e+01],
          [-1.5302700e+01,  1.4431200e+00, -4.7971500e+01],
          [-6.0784298e-01, -1.2160700e+01,  4.3689098e+01],
          [ 1.7079800e+01,  1.9666600e+00, -8.3763802e+01],
          [-4.1084499e+00,  3.5197800e+01, -2.3239799e+01]]],


        [[[-6.8052673e-01,  4.3509445e+00, -1.5487452e+00],
          [ 5.8407185e+01, -8.2160988e+01, -1.2327696e+01],
          [-9.1078697e+01, -4.6658367e+01, -7.3403926e+00],
          [ 9.6684196e+01,  1.5651445e+01,  2.6858237e+00],
          [-7.8112083e+01,  6.0135139e+01, -6.0886226e+00],
          [ 3.1481721e+00,  9.0496506e+01, -1.1837043e+01],
          [-1.9831656e+01, -8.0089989e+01, -2.5694869e+00],
          [ 5.0045956e+01,  5.1196590e+01, -1.4792501e+01],
          [ 2.3459833e+01, -4.9363476e+01,  1.9859182e+01]]]]],
      )

In [48]:
keypoints_array.shape

(1, 8, 1, 9, 3)

In [49]:
keypoints = tf.convert_to_tensor(keypoints_array, dtype=tf.float32)

In [50]:
camera_matrix_array = np.array([[[572.4114 ,   0.     , 325.2611 ],
        [  0.     , 573.57043, 242.049  ],
        [  0.     ,   0.     ,   1.     ]]])

In [51]:
camera_matrix_array.shape

(1, 3, 3)

In [52]:
camera_matrix = tf.convert_to_tensor(camera_matrix_array, dtype=tf.float32)

In [53]:
keypoints

<tf.Tensor: shape=(1, 8, 1, 9, 3), dtype=float32, numpy=
array([[[[[-9.7732497e-03,  3.6659201e-03, -1.4534000e-03],
          [ 3.6046398e+01, -1.4680500e+01, -4.5020599e+01],
          [-3.0289101e+01, -7.2402501e+00, -4.2632900e+01],
          [ 7.4425201e+00,  2.3966400e+01, -3.9362701e+01],
          [-4.3485899e+00,  3.6769500e+00,  4.5835800e+01],
          [ 6.2882501e-01, -3.6412800e+01, -2.7732599e+01],
          [-2.6754901e-01,  3.7588799e+01, -4.7640200e+00],
          [ 3.0029400e+01, -2.3939800e+01, -8.1097898e+00],
          [-2.8789900e+01, -1.9449200e+01, -9.0417604e+00]]],


        [[[ 1.6300200e-02, -2.3040799e-03, -1.1291500e-02],
          [ 5.5248199e+00,  5.4157101e+01, -9.6322701e+01],
          [-4.1018100e+00,  1.2732400e+01,  9.6678497e+01],
          [-9.1580000e+00, -4.1244202e+01, -8.7472397e+01],
          [ 7.3375401e+00,  9.0886101e+01, -1.1365300e+01],
          [-1.0262200e+01, -9.0547600e+01, -3.7563899e-01],
          [-4.7794201e+01,  1.6508699e+

In [54]:
camera_matrix

<tf.Tensor: shape=(1, 3, 3), dtype=float32, numpy=
array([[[572.4114 ,   0.     , 325.2611 ],
        [  0.     , 573.57043, 242.049  ],
        [  0.     ,   0.     ,   1.     ]]], dtype=float32)>

In [55]:
#img_batch[4]

In [93]:
#img_batch[5]

<tf.Tensor: shape=(1, 3, 3), dtype=float32, numpy=
array([[[572.4114 ,   0.     , 325.2611 ],
        [  0.     , 573.57043, 242.049  ],
        [  0.     ,   0.     ,   1.     ]]], dtype=float32)>

In [56]:
#keypoints = img_batch[4]
#camera_matrix = img_batch[5]

In [57]:
testingdata_iterator = iter(testing_images)
runnetwork(testingdata_iterator, int(test_batches), keypoints, camera_matrix, no_objects + 1)

Instructions for updating:
Use fn_output_signature instead
[TensorShape([8, 9, 2]), TensorShape([8, 9, 3]), TensorShape([3, 3])]


2023-04-06 19:32:11.447711: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2023-04-06 19:32:14.403761: I tensorflow/stream_executor/cuda/cuda_blas.cc:1804] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-06 19:32:14.693088: I tensorflow/compiler/xla/service/service.cc:170] XLA service 0x7efc7c8566b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-06 19:32:14.693138: I tensorflow/compiler/xla/service/service.cc:178]   StreamExecutor device (0): NVIDIA A10G, Compute Capability 8.6
2023-04-06 19:32:14.765468: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:263] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-06 19:32:15.586602: I tensorflow/compiler/jit/xla_compilation_cache.cc:481] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-04-06 19:32:19.4989

-0.925360084
tf.print= [[[[-0.0578267425 0.0547863767 0.00218745857 ... -0.110988475 -0.000841193 -0.152722359]
   [-0.0369873233 0.134331346 0.0189906303 ... -0.0904605538 0.0147983702 -0.157904372]
   [-0.0145833129 0.150997877 0.00718277181 ... -0.0358056 0.0343351588 -0.119491696]
   ...
   [0.152773753 0.17745173 -0.207866713 ... 0.0678132251 -0.166177988 0.0776503384]
   [0.125595823 0.15193294 -0.231210098 ... 0.0662854463 -0.127840087 0.137255266]
   [-0.0352426656 0.227821022 -0.0372289643 ... -0.0525983498 0.0835434794 -0.0571290068]]

  [[-0.0704033822 0.150795236 0.145627499 ... -0.170981735 -0.00872801058 -0.30433616]
   [0.159804344 0.298555344 0.0928240642 ... -0.126922444 -0.0848130211 -0.222648308]
   [0.209027365 0.319793165 0.0829005912 ... -0.100691631 -0.0721257776 -0.173886091]
   ...
   [0.164129212 0.145432428 -0.153351083 ... 0.0345689096 -0.0756454766 0.174294278]
   [0.0919490531 0.053765431 -0.186145604 ... 0.0341878869 -0.120431535 0.252732843]
   [-0.09200